In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load your trained YOLOv8 model
model = YOLO('best.pt')  # Replace with 'yolov8n.pt' if needed

# Load corner points calibration data
corner_points = np.load('corner_points.npz')['corner_points']
tl, tr, bl, br = corner_points[:4]

# Define class names (must match your trained model)
class_names = ['AA', 'D type', '9v']

# Initialize camera (adjust camera_index as needed)
camera_index = 1  # Example: change this to match your external camera index
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print(f"Error: Could not open camera with index {camera_index}")
    exit()

# Function to map detected position to normalized grid coordinates
def map_to_grid(x, y, tl, tr, bl, br):
    # Compute vectors for mapping to normalized grid coordinates
    v1 = np.array([tr[0] - tl[0], tr[1] - tl[1]])
    v2 = np.array([bl[0] - tl[0], bl[1] - tl[1]])
    vp = np.array([x - tl[0], y - tl[1]])

    # Calculate normalized grid coordinates (u, v)
    u = np.dot(vp, v1) / np.dot(v1, v1)
    v = np.dot(vp, v2) / np.dot(v2, v2)

    return u, v

# Function to convert grid coordinates to robot coordinates
def grid_to_robot(u, v, tl, tr, bl, br):
    # Calculate the robot coordinates (X, Y, Z) from the normalized grid coordinates (u, v)
    x_robot = tl[0] + u * (tr[0] - tl[0]) + v * (bl[0] - tl[0])
    y_robot = tl[1] + u * (tr[1] - tl[1]) + v * (bl[1] - tl[1])
    z_robot = tl[2] + u * (tr[2] - tl[2]) + v * (bl[2] - tl[2])  # Adjust for height if necessary

    return x_robot, y_robot, z_robot

try:
    while True:
        ret, frame = cap.read()  # Capture frame-by-frame

        if not ret:
            print("Error: Failed to capture image")
            break

        # Perform inference
        results = model(frame)[0]  # Get the first result

        # Iterate through the detected objects
        detections = []
        for result in results.boxes.data:
            x1, y1, x2, y2, confidence, class_id = result.tolist()

            # Calculate center coordinates of the bounding box
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2

            # Map the detected position to the grid coordinates
            u, v = map_to_grid(x_center, y_center, tl, tr, bl, br)

            # Convert grid coordinates to robot coordinates
            x_robot, y_robot, z_robot = grid_to_robot(u, v, tl, tr, bl, br)

            # Save detection information (battery type and coordinates)
            detections.append({
                'battery_type': class_names[int(class_id)],
                'robot_coordinates': (x_robot, y_robot, z_robot)
            })

            # Draw bounding box and label
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            label = f'{class_names[int(class_id)]} {confidence:.2f}'
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Display the frame with detections
        cv2.imshow('Battery Detection', frame)

        # Print or log the detected battery information
        for detection in detections:
            print(f"Detected {detection['battery_type']} at robot coordinates {detection['robot_coordinates']}")

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Detection stopped by user.")
finally:
    # Release the capture and close all windows
    cap.release()
    cv2.destroyAllWindows()



0: 480x640 1 D type, 490.9ms
Speed: 15.6ms preprocess, 490.9ms inference, 45.8ms postprocess per image at shape (1, 3, 480, 640)
Detected D type at robot coordinates (334.35654897315, 226.9064925997726, 149.03160248598783)

0: 480x640 1 D type, 324.9ms
Speed: 7.9ms preprocess, 324.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected D type at robot coordinates (333.6454114370375, 226.94887613135924, 149.00014018618717)

0: 480x640 1 D type, 294.5ms
Speed: 6.1ms preprocess, 294.5ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
Detected D type at robot coordinates (333.63142898159265, 227.10101809266308, 149.00138138567215)

0: 480x640 1 D type, 275.9ms
Speed: 3.7ms preprocess, 275.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected D type at robot coordinates (333.6162907570555, 227.01336351278712, 148.9996231384056)

0: 480x640 1 D type, 282.5ms
Speed: 6.1ms preprocess, 282.5ms inference, 1.0ms postprocess per image

In [2]:
import time

def move_to_position(x, y, z):
    # Simulate the robot's movement in RC software
    print(f"Moving to position: X: {x}, Y: {y}, Z: {z}")
    time.sleep(3)  # Simulate delay

def magnet_control(state):
    # Simulate magnet control in RC software
    if state == "ON":
        print("Magnet ON (Simulation)")
    else:
        print("Magnet OFF (Simulation)")
    time.sleep(3)  # Simulate delay

def main_simulation():
    # Corner points simulation for battery sorting
    aa_drop_position = (100, 200, 300)  # Example drop position for AA
    d_drop_position = (150, 250, 350)   # Example drop position for D
    v9_drop_position = (200, 300, 400)  # Example drop position for 9V
    
    # Simulate detecting a battery
    print("Battery detected: AA")
    
    # Move to the battery pickup position (simulate)
    move_to_position(50, 50, 50)
    
    # Activate the electromagnet (simulate)
    magnet_control("ON")
    
    # Move to the AA drop position (simulate)
    move_to_position(*aa_drop_position)
    
    # Deactivate the electromagnet (simulate)
    magnet_control("OFF")

if __name__ == "__main__":
    main_simulation()


Battery detected: AA
Moving to position: X: 50, Y: 50, Z: 50
Magnet ON (Simulation)
Moving to position: X: 100, Y: 200, Z: 300
Magnet OFF (Simulation)


In [5]:
import cv2
import socket
import time
import torch

# Load your trained YOLOv8 model
model = YOLO('best.pt')  # Replace with 'yolov8n.pt' if needed

# Define class names (must match your trained model)
class_names = ['AA', 'D type', '9v']

# Setup TCP connection parameters (change according to mode)
HOST = '127.0.0.1'  # Localhost or IP of real robot
PORT = 2000         # Port number for TCP/IP communication

# Define function to send commands to robot (simulation or real)
def send_command(command):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(command.encode())
        data = s.recv(1024)
        print(f'Received: {data.decode()}')

# Define function to capture frames and detect batteries in real time
def detect_batteries():
    cap = cv2.VideoCapture(1)  # Open the default camera
    
    while True:
        ret, frame = cap.read()  # Capture frame-by-frame
        if not ret:
            break

        # Perform YOLO detection
        results = model(frame)
        results.render()  # Render the results on the frame

        # Display the resulting frame with detections
        cv2.imshow('Battery Detection', frame)

        # Extract battery type and coordinates
        for result in results.xyxy[0]:
            x1, y1, x2, y2, confidence, class_id = result
            battery_type = int(class_id)
            battery_coordinates = (x1.item(), y1.item(), x2.item(), y2.item())
            
            if battery_type == 0:  # Assuming 0 is AA type
                move_robot_to_battery(battery_coordinates, 'AA')
            elif battery_type == 1:  # Assuming 1 is 9V type
                move_robot_to_battery(battery_coordinates, '9V')
            elif battery_type == 2:  # Assuming 2 is D type
                move_robot_to_battery(battery_coordinates, 'D')

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Define function to move the robot to the battery position based on type
def move_robot_to_battery(coords, battery_type):
    x1, y1, x2, y2 = coords
    # Calculate battery center point
    x = (x1 + x2) / 2
    y = (y1 + y2) / 2

    # Move robot to the detected battery's position
    send_command(f'MOVE X={x}, Y={y}')
    time.sleep(3)  # Wait for robot to reach

    # Adjust height based on battery type
    if battery_type == 'AA':
        send_command('MOVE Z=1')  # Move 1 cm above
    elif battery_type == '9V':
        send_command('MOVE Z=2')  # Move 2 cm above
    elif battery_type == 'D':
        send_command('MOVE Z=4')  # Move 4 cm above
    
    time.sleep(3)  # Delay for smoothness

    # Activate electromagnet to pick the battery
    send_command('MAGNET ON')
    time.sleep(3)  # Wait to pick

    # Move to drop location based on battery type
    if battery_type == 'AA':
        send_command('MOVE X=DROP_AA_X, Y=DROP_AA_Y')
    elif battery_type == '9V':
        send_command('MOVE X=DROP_9V_X, Y=DROP_9V_Y')
    elif battery_type == 'D':
        send_command('MOVE X=DROP_D_X, Y=DROP_D_Y')

    time.sleep(3)  # Wait to reach the drop point

    # Deactivate electromagnet to drop the battery
    send_command('MAGNET OFF')
    time.sleep(3)  # Wait for the battery to detach

# Main function to run both detection and robot control
def main():
    print("Starting battery detection and robot control...")
    
    # Start the battery detection and robot movement
    detect_batteries()

if __name__ == '__main__':
    main()


Starting battery detection and robot control...

0: 480x640 1 D type, 1 9v, 355.4ms
Speed: 6.5ms preprocess, 355.4ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)


AttributeError: 'list' object has no attribute 'render'